In [ ]:
import os
import torch
import matplotlib.pyplot as plt

# Util function for loading meshes
from pytorch3d.io import load_objs_as_meshes, load_obj

# Data structures and functions for rendering
from pytorch3d.structures import Meshes
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras, 
    PointLights, 
    Materials, 
    RasterizationSettings, 
    MeshRenderer, 
    MeshRasterizer,  
    SoftPhongShader,
    TexturesVertex,
)

# Setup
device = torch.device("cuda:0")
torch.cuda.set_device(device)

# Set paths
DATA_DIR = "/home/borth/2d-gaussian-splatting/data/ShapeNetCoreTiny/02691156/1a04e3eab45ca15dd86060f189eb133/models"
obj_filename = os.path.join(DATA_DIR, "model_normalized.obj")
mesh = load_objs_as_meshes([obj_filename], device=device)

In [ ]:
from neural_poisson.data.prepare import (
    get_points_camera_space_attributes,
    get_depth_camera_space_attributes,
    compute_half_pixel_princial_point,
    compute_focal_length_from_FoV,
    rasterize_attributes,
    depth_to_normals,
    depth_to_points,
    extract_surface_data,
    uniform_sphere_cameras,
)

# initialize a camera and rasterizer
image_size = 512
segments = 20
cameras = uniform_sphere_cameras(segments=segments)

elev = 8
azim = 1
data = extract_surface_data(
    camera=cameras[elev + azim * segments],
    mesh=mesh,
    image_size=image_size,
)

plt.imshow(data["normal_map"][0].detach().cpu().numpy())

In [ ]:
import open3d as o3d

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(data["points"].detach().cpu().numpy())
pcd.normals = o3d.utility.Vector3dVector(data["normals"].detach().cpu().numpy())
o3d.visualization.draw_plotly([pcd])
